In [2]:
# Imports, as always...
from os import listdir, makedirs, path
from tqdm.notebook import tqdm
import numpy as np
import pickle
import torch

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [5]:
# Random seeds.
torch.manual_seed(42)
np.random.seed(42)

# Fingerprint Classification

This notebook aims to replicate and advance the approaches to classification in [Martina et al. (2021)](https://arxiv.org/pdf/2109.11405), in which an SVM is used to binarily classify whether a given classical measurement was or was not produced by a given quantum circuit.

This is a simple task which, when taken together with their *very* small circuit, yields for near perfect accuracy. Here, we will complicate things to push the capability of the models to investigate more thoroughly what may or may not be possible with regard to classifying the membership of a quantum state to a quantum device by its "noise fingerprint". 

The ideas fitting into the work of this notebook are as follows:
- *Multi-class classification*. Using the data produced by Martina et al. (2021), can we present a multi-class prediction model that is not given any bias towards any particular model -- given a measurement of a quantum state, which device produced it?
- *Larger/deeper circuits*. How does the performance degrade as the number of qubits increases, or as the circuit depth increases?
- *Noise severity analysis*. Under which severities/forms of noise is performance best? Ideally, we can produce a visualisation of performance (e.g. accuracy) vs. noise intensity/severity. We might expect poor performance with little/no noise (not enough distinguishing information between membership classes), good performance with moderate noise, then poor performance again with large amounts of noise (too much randomness).

For clarity, 'membership to a quantum device' in this context refers to 'being produced by that device'.

## Martina et al. (2021)'s Dataset

Only the raw data is given, and the `createDataset.py` and `extractExecuction.py` scripts are abhorrent messes and crimes against humanity, so I'll do my best to re-create the "extracting" and "creating" process for a dataset from the data on their [GitHub](https://github.com/trianam/learningQuantumNoiseFingerprint/tree/main).

In [6]:
# List all files in the "walker" directory.
file_list = listdir('./martina/data/walker')

# List of machines.
# Note: IBM Bogota's files cannot be read -- ALL lead to pickle underflow.
machines = ['ibmq_athens', 'ibmq_casablanca', 'ibmq_lima', 'ibmq_quito', 'ibmq_santiago', 'ibmq_5_yorktown']

# How many files does each machine have.
counts = {machine : 0 for machine in machines}
for file in file_list:
    for machine in machines:
        if machine in file:
            counts[machine] += 1

# Any count above 250 includes custom splits -- we're not too interested in those.
display(counts)

{'ibmq_athens': 750,
 'ibmq_casablanca': 500,
 'ibmq_lima': 250,
 'ibmq_quito': 250,
 'ibmq_santiago': 250,
 'ibmq_5_yorktown': 250}

### Extracting

This is the process of reading the stored data and translating it into probability distributions.

In [7]:
# File paths.
base_path = './martina/data/walker'
extracted_path = './martina/data/walkerExtracted'
    
# Generate the output path.
makedirs(extracted_path, exist_ok=True)

# "Window sizes".
ks = [1000]

In [95]:
# Helper function to filter the file list into the (non-split) files of only a given machine.
def filter_to_machine(file_list, machine):
    # List of words that specify different types of data (e.g. split).
    no_words = ['split', 'bis']
    
    return filter(
        lambda file : machine in file and not (any([word in file for word in no_words])), file_list
    )

In [96]:
# Extracting executions for each machine.
for machine in tqdm(machines, desc='Extracting'):
    executions = []
    
    # For each file belonging to the current machine.
    for file in tqdm(filter_to_machine(file_list, machine), desc='Reading files', total=250):
        # Read the contents of the file.
        contents = pickle.load(open(path.join(base_path, file), 'rb'))
        
        # For each run of the circuit (of which there are 8000 in the Martina paper and data).
        for n in range(len(contents['results'][0]['data']['memory'])):
            current_execution = []
            
            # Note: we will not be doing repeated measures, nor will we "read all bits".
            
            # For each measurement step t (of which there are 9 in the Martina paper and data).
            for t in range(len(contents['results'])):
                execution = int(contents['results'][t]['data']['memory'][n], 0)
                current_execution.append(execution)

            executions.append(current_execution)
            
    # Cast to numpy array.
    executions = np.array(executions)
    
    # Save the full executions of this machine. 
    np.savetxt(path.join(extracted_path, f'{machine}-executions.csv'), executions)
    
    # Break the executions into windows to be saved.
    for k in ks:
        # The window size must cleanly divide the number of executions.
        if executions.shape[0] % k != 0: raise(Exception('Indivisible by window size.'))
        
        # Initialise probabilities array.
        probs = np.zeros(shape=(
            executions.shape[0] // k, executions.shape[1], np.unique(executions).shape[0]
        ), dtype=np.float32)
        
        # Calculate probabilities with the given window size.
        for n in tqdm(range(executions.shape[0]), desc='Calculating probabilities'):
            i = n // k
            
            for t in range(executions.shape[1]):
                probs[i, t, executions[n, t]] += 1
                
        probs = probs / k
        
        # Save the window.
        np.save(path.join(extracted_path, f'{machine}-probabilities-{k}.npy'), probs)

Extracting:   0%|          | 0/6 [00:00<?, ?it/s]

Reading files:   0%|          | 0/250 [00:00<?, ?it/s]

Calculating probabilities:   0%|          | 0/2000000 [00:00<?, ?it/s]

Reading files:   0%|          | 0/250 [00:00<?, ?it/s]

Calculating probabilities:   0%|          | 0/2000000 [00:00<?, ?it/s]

...

Calculating probabilities:   0%|          | 0/2000000 [00:00<?, ?it/s]

Reading files:   0%|          | 0/250 [00:00<?, ?it/s]

Calculating probabilities:   0%|          | 0/2000000 [00:00<?, ?it/s]

Reading files:   0%|          | 0/250 [00:00<?, ?it/s]

Calculating probabilities:   0%|          | 0/2000000 [00:00<?, ?it/s]

### Creating

Now we arrange the extracted run statistics into a dataset. For the sake of ease of use, we'll chuck all that into a PyTorch Dataset and then Dataloader.

In [8]:
# File paths.
dataset_path = './martina/data/walkerDataset'
makedirs(dataset_path, exist_ok=True)

# Specify the window size.
k = ks[0]

In [9]:
# Pack all the probability distributions (from all machines) into an array.
probs = [np.load(path.join(extracted_path, f'{machine}-probabilities-{k}.npy')) for machine in machines]
order = [np.arange(prob.shape[0]) for prob in probs]

# Features (x), labels (y) format.
xs, ys = [], []
for i in range(min(map(len, order))):
    for p in range(len(probs)):
        xs.append(probs[p][order[p][i]])
        ys.append(p)
        
# Numpify those arrays.
xs = np.array(xs, dtype=np.float32)
ys = np.array(ys, dtype=np.float32)

# Save in this format.
np.savez_compressed(path.join(dataset_path, f'all-dataset-{k}'))

In [10]:
# PyTorch Dataset class for handling these data.
class PyTorchDataset(Dataset):
    def __init__(self, xs : np.array, ys : np.array):
        self.xs = xs
        self.ys = ys
        
    def __len__(self):
        return ys.shape[0]
    
    def __getitem__(self, idx):
        return xs[idx], ys[idx]
    
# Train-test-validation splitting the data.
test_size, val_size = 0.25, 0.25
xs_train, xs_test, ys_train, ys_test = train_test_split(xs, ys, test_size=test_size, shuffle=False)
xs_train, xs_val, ys_train, ys_val = train_test_split(xs_train, ys_train, test_size=val_size, shuffle=False)

# Create train-test-val Dataset objects.
train_dataset = PyTorchDataset(xs_train, ys_train)
val_dataset = PyTorchDataset(xs_val, ys_val)
test_dataset = PyTorchDataset(xs_test, ys_test)

# And create train-test-val Dataloader objects.
batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [11]:
# e.g.
train_dataset[0]

(array([[0.464, 0.53 , 0.002, 0.004],
        [0.236, 0.253, 0.247, 0.264],
        [0.076, 0.466, 0.204, 0.254],
        [0.153, 0.446, 0.159, 0.242],
        [0.139, 0.299, 0.189, 0.373],
        [0.251, 0.205, 0.221, 0.323],
        [0.297, 0.203, 0.33 , 0.17 ],
        [0.396, 0.23 , 0.223, 0.151],
        [0.377, 0.271, 0.206, 0.146]], dtype=float32),
 0.0)

## Multi-class Classification

Given the measurement of a quantum state, what is the probability distribution over the set of devices (for the likelihood of membership), and subsequently which device is most likely to have produced the state?

The paper has two considerations for the input: (1) considering measurement outcomes at a single measurement step $k\in[1,\dots,9]$; and (2) concatenating all measurement outcomes in an ordered series $1,\dots,k$. Since the circuit is supposedly regenerated and then measured independently for each measurement step $k$, the only tangible difference between them is *time*. This is the point, actually -- the elements of the ordered series are independent of one another, yet ordered in time. 

### Single-measurement

We'll define a model then train it to classify a given probability distribution as belonging to one of the machines. We'll train on each of the $k$ to obtain $k$ models that have learnt to discriminate distributions by machine membership at each time step. Then we can use each of these models to test on each time step too, giving that we will yield $k$ accuracy scores per model -- how well does each model discriminate at a given time step, considering the time step it was trained in. 

If we find that models are only really capable of classifying in or very near the time step they were trained in, then we are discovering that time-ordered series of noise are necessary to distinguish machines. This was the broad conclusion of the paper, but their method was not as direct as this (for whatever reason). On the other hand, having a model trained at the $k$-th step being able to classify quite well at the ($k+k')$-th for some $k'\gg0$ tells us that there is enough information in a slice of time to understand the inherent difference between the noise being produced by machines -- time-dependent or otherwise.

Another thing that might be interesting is to train a model with *all* data with no regard to time. This may be considered a brute-force version of the aforementioned test for time-dependence, but I think this might lead to a model capable of doing well at time steps it has never seen. For the sake of this last point, we might like to only train on the first few time steps (say 5) and evaluate on the final time step (i.e. 9) to allow for some distance between train and test sets.

Finally, we may also find interesting resutls training a model to discriminate *the time step* rather than the machine. That is, given a probability distribution from a single machine, can we classify the time step in which it was generated. We might then find use for this classify as a "selector" of sorts for choosing better architectural choice later on (e.g. if we have $k$ different classifiers for each time step as suggested above, then this model could be used to decide which to use given an unknown state).